In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import numpy as np
import pandas as pd
import re 
import pytest
import sys
import re
np.set_printoptions(suppress=True)
pd.options.display.float_format = "{:,.2f}".format
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
import best_subset as bs_pkgs


In [4]:

def process_exaustive_and_return_top(df, features, return_top=100):
    mask = df['Models'].notna()  # Ensure we exclude NaN values    
    for feature in features:
        mask &= df['Models'].str.contains(rf'\b{feature}\b', regex=True, na=False)
    top =  df[mask] 

    top['rank'] = top.groupby("Var Number")['Scores'].rank(ascending=False)
    top = top[top['rank']<=100]
    top.drop('rank', axis=1, inplace=True)
    top = top.reset_index(drop=True)
    return top


def check_if_features_in(df, features):
    mask = df['Models'].notna()  # Ensure we exclude NaN values    
    for feature in features:
        mask &= df['Models'].str.contains(rf'\b{feature}\b', regex=True, na=False)
    return df[mask]

def compare_dataframes(df1: pd.DataFrame, df2: pd.DataFrame):
    """Compares two pandas DataFrames, rounding floating-point columns to 2 decimal places."""
    float_cols = df1.select_dtypes(include=['float']).columns
    df1_rounded = df1.copy()
    df2_rounded = df2.copy()
    df1_rounded[float_cols] = df1_rounded[float_cols].round(2)
    df2_rounded[float_cols] = df2_rounded[float_cols].round(2)
    pd.testing.assert_frame_equal(df1_rounded, df2_rounded, check_dtype=False)
    print("Dataset Match")

def order_models_field(df):
    df['Models'] = df['Models'].apply(
        lambda model: " ".join(
            sorted(model.split(" "), key=lambda x: int(re.search(r'\d+', x).group()))
        )
    )
    df = df.reset_index(drop=True)
    return df 

def order_models_filed_all(df):
    df['Models'] = df['Models'].apply(lambda model: " ".join(sorted(model.split(" "))) )
    return df


def create_synthetic_data(seed=42, n=50000, p=15):
    """
    Creates a DataFrame X of shape (n, p+1) with columns:
      - 'const': all ones (intercept)
      - 'x1', 'x2', ... 'x15'
    And a Series y with binary (0/1) outcomes drawn from a logistic model.
    
    Some of the 15 features have nonzero coefficients, others are zero,
    so there's meaningful signal to detect in a logistic regression.
    """

    np.random.seed(seed)

    # 1) Generate random features ~ N(0,1)
    X_base = np.random.randn(n, p)
    
    # 2) Define "true" coefficients
    #    For instance, let's say 5 features matter:
    #    x1, x2, x3, x4, x5 have some nonzero betas.
    #    The remaining x6..x15 have 0 effect.
    betas_true = np.array([1.5, -2.0, 0.75, 1.25, -0.5] + [0]*(p-5))
    #     -> length = 15
    
    # 3) Linear predictor: X_base dot betas_true
    #    shape -> (n, )
    lin_pred = X_base.dot(betas_true)
    
    # 4) Convert linear predictor to probability via logistic function
    #    p_i = 1 / (1 + exp(-lin_pred))
    prob = 1.0 / (1.0 + np.exp(-lin_pred))
    
    # 5) Draw binary outcomes y from Bernoulli(prob)
    y_vals = np.random.binomial(1, prob)
    
    # 6) Create a DataFrame for features, plus an intercept column
    df = pd.DataFrame(X_base, columns=[f"x{i+1}" for i in range(p)])
    df.insert(0, "const", 1.0)  # intercept
    
    # 7) Create a Series for y
    y = pd.Series(y_vals, name="y")
    
    return df, y


def create_synthetic_data_logistic(seed=42, n=50000, p=15):
    """
    Creates a DataFrame X of shape (n, p+2) with columns:
      - 'const': all ones (intercept)
      - 'x1', 'x2', ... 'x15'
      - 'weight': randomly generated weights between 0 and 100
    And a Series y with binary (0/1) outcomes drawn from a logistic model.
    
    Some of the 15 features have nonzero coefficients, others are zero,
    so there's meaningful signal to detect in a logistic regression.
    """

    np.random.seed(seed)

    # 1) Generate random features ~ N(0,1)
    X_base = np.random.randn(n, p)
    
    # 2) Define "true" coefficients
    #    For instance, let's say 5 features matter:
    #    x1, x2, x3, x4, x5 have some nonzero betas.
    #    The remaining x6..x15 have 0 effect.
    betas_true = np.array([1.5, -2.0, 0.75, 1.25, -0.5] + [0]*(p-5))
    #     -> length = 15
    
    # 3) Linear predictor: X_base dot betas_true
    #    shape -> (n, )
    lin_pred = X_base.dot(betas_true)
    
    # 4) Convert linear predictor to probability via logistic function
    #    p_i = 1 / (1 + exp(-lin_pred))
    prob = 1.0 / (1.0 + np.exp(-lin_pred))
    
    # 5) Draw binary outcomes y from Bernoulli(prob)
    y_vals = np.random.binomial(1, prob)
    
    # 6) Create a DataFrame for features, plus an intercept column
    df = pd.DataFrame(X_base, columns=[f"x{i+1}" for i in range(p)])
    df.insert(0, "const", 1.0)  # intercept

    # 7) Generate weights between 0 and 100
    weights = np.random.uniform(0, 100, size=n)
    df['weight'] = weights  # Add 'weight' column
    
    # 8) Create a Series for y
    y = pd.Series(y_vals, name="y")
    
    return df, y


def create_synthetic_data_linear_regression(seed=42, n=50000, p=15, sigma=1.0):
    """
    Creates synthetic data for linear regression.

    Args:
        seed: Random seed for reproducibility.
        n: Number of samples.
        p: Number of features (excluding the intercept).
        sigma: Standard deviation of the error term.

    Returns:
        Tuple: A DataFrame `df` containing the features (including 'const' and 'weight') 
               and a Series `y` representing the target variable.
    """

    np.random.seed(seed)

    # 1) Generate random features ~ N(0,1)
    X_base = np.random.randn(n, p)

    # 2) Define "true" coefficients (including the intercept)
    #    Let's say 5 features have a non-zero effect.
    betas_true = np.array([2.0, 1.5, -2.0, 0.75, 1.25, -0.5] + [0] * (p - 5))
    # betas_true now includes the intercept (e.g., 2.0) in the first position.

    # 3) Generate weights between 0 and 100
    weights = np.random.uniform(0, 100, size=n)

    # 4) Create a DataFrame for features, plus an intercept column
    df = pd.DataFrame(X_base, columns=[f"x{i + 1}" for i in range(p)])
    df.insert(0, "const", 1.0)  # intercept

    # 5) Linear predictor: X dot betas_true (including intercept)
    #    shape -> (n, )
    lin_pred = df.drop(columns=['const']).dot(betas_true[1:]) + betas_true[0] # Account for intercept in betas_true

    # 6) Generate the target variable y with added noise:
    #    y = linear predictor + error
    #    where error ~ N(0, sigma^2)
    y_vals = lin_pred + np.random.normal(0, sigma, size=n)

    # 7) Add 'weight' column to DataFrame
    df['weight'] = weights

    # 8) Create a Series for y
    y = pd.Series(y_vals, name="y")

    return df, y


def create_synthetic_data_ordinal_logit(seed=42, n_samples=50000, n_features=15, n_classes=3, 
                                      beta_scale=1.0, class_separation=1.0):
    """
    Creates synthetic data for ordinal logistic regression.

    Args:
        seed: Random seed for reproducibility
        n_samples: Number of observations
        n_features: Number of features (excluding intercept)
        n_classes: Number of ordinal classes (3-5 recommended)
        beta_scale: Scale factor for coefficient magnitudes
        class_separation: Controls spread between cutpoints

    Returns:
        Tuple: (df, y) where df contains features + weights, y contains ordinal labels
    """
    np.random.seed(seed)
    
    # 1. Generate features matrix with intercept
    X_base = np.random.randn(n_samples, n_features)
    df = pd.DataFrame(X_base, columns=[f"x{i+1}" for i in range(n_features)])
    df.insert(0, "const", 1.0)

    # 2. Generate true parameters
    n_cutpoints = n_classes - 1
    
    # Coefficients (first 5 features have non-zero effects)
    beta_true = np.zeros(n_features + 1)  # +1 for intercept
    beta_true[0] = 1.0  # Intercept
    beta_true[1:6] = np.array([1.5, -2.0, 0.75, 1.25, -0.5]) * beta_scale
    
    # Cutpoints (sorted for identifiability)
    theta_true = np.sort(np.random.randn(n_cutpoints) * class_separation)

    # 3. Compute linear predictor
    X_mat = df.values
    XB = X_mat @ beta_true  # Shape (n_samples,)

    # 4. Calculate class probabilities using proportional odds model
    z = theta_true[:, None] - XB  # Shape (n_cutpoints, n_samples)
    cumulative_probs = 1 / (1 + np.exp(-z))  # CDF values
    
    # Pad with 0 (left) and 1 (right) for class probabilities
    padded_probs = np.vstack([np.zeros((1, n_samples)),
                             cumulative_probs,
                             np.ones((1, n_samples))])
    
    # Calculate class probabilities via differences
    class_probs = np.diff(padded_probs, axis=0)  # Shape (n_classes, n_samples)
    class_probs = class_probs.T  # Shape (n_samples, n_classes)

    # 5. Generate ordinal labels
    u = np.random.rand(n_samples)
    cumulative_probs = np.cumsum(class_probs, axis=1)
    y = (u[:, None] < cumulative_probs).argmax(axis=1)

    # 6. Add weights and return
    df["weight"] = np.random.uniform(0, 100, size=n_samples)
    return df, pd.Series(y, name="y")
# Example usage
if __name__ == "__main__":
    df, y = create_synthetic_data_logistic(p=15)
    # print(df.head())
    # print(y.head())
    print(df.columns)
    print("df shape:", df.shape)
    print("y shape:", y.shape)

Index(['const', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10',
       'x11', 'x12', 'x13', 'x14', 'x15', 'weight'],
      dtype='object')
df shape: (50000, 17)
y shape: (50000,)


# Test 1

* Compare Exaustive results without `weights` using forced features

## Run Exhaustive Model. Use it as Baseline

In [230]:

df, y = create_synthetic_data_logistic(p=15)
df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x15": "gt_fico"}, inplace=True)
# df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x5": "gt_fico"}, inplace=True)
candidates = df.columns.tolist()
candidates.remove('const')
candidates.remove('weight')
# candidates = df.columns[1:-1].tolist()
results =  bs_pkgs.best_subset_exhaustive(df, y, candidates, method='logistic')

Finished Var Family: 1
Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
Finished Var Family: 13
Finished Var Family: 14
Finished Var Family: 15
Total Models: 32767


# Test 1: Compare exhaustive and new algo

# Filter Exhaustive

* filter models with these features `['x1','x2', '*fico*']`. Must have x1, x2 and at least 1 fico feature (for example)
* Return top 100 models given the conditions above

In [263]:
def check_if_features_in(df, features):
    mask = df['Models'].notna()  # Ensure we exclude NaN values    
    for feature in features:
        feature = feature.strip()  # Remove any accidental whitespace

        if feature.startswith("*") and feature.endswith("*"):
            feature = feature.strip("*")  # Remove * to get the base word
            pattern = rf"\b{feature}\b|{feature}"  # Match exact word or substring
        
        elif feature.startswith("*"):
            feature = feature.strip("*")
            pattern = rf"\b{feature}\b|{feature}$"  # Match exact word or ending

        elif feature.endswith("*"):
            feature = feature.strip("*")
            pattern = rf"\b{feature}\b|^{feature}"  # Match exact word or start

        else:
            pattern = rf"\b{feature}\b"  # Exact word match
        
        mask &= df['Models'].str.contains(pattern, case=False, regex=True, na=False)
    return df[mask]
# exhaustive_filter_ = check_if_features_in(results, ['x1', 'x2' , "*fico"])
# print(a_bfs_v2.shape, check_if_features_in(a_bfs_v2, features).shape)
# exhaustive_filter_ = check_if_features_in(results, ['x14','x15', '*fico*'])
# exhaustive_filter_ = check_if_features_in(results, ['x1','x9','x12' ])
exhaustive_filter_ = check_if_features_in(results, ['x1', 'x2' , "*fico"])
# forced_vars=['x1', 'x2' , "*fico*"]
# exhaustive_filter_ = check_if_features_in(results, [])


exhaustive_filter_ = order_models_filed_all(exhaustive_filter_)
exhaustive_filter_['rank'] = exhaustive_filter_.groupby("Var Number")['Scores'].rank(ascending=False)
exhaustive_filter_ = exhaustive_filter_[exhaustive_filter_['rank']<=100]
exhaustive_filter_.drop('rank', axis=1, inplace=True)
exhaustive_filter = exhaustive_filter_.reset_index(drop=True)
# print(exhaustive_filter.to_markdown())
exhaustive_filter

,Var Number,Models,Scores
0,3,fico x1 x2,"18,375.78"
1,3,gt_fico x1 x2,"16,810.12"
2,4,fico fico_lt x1 x2,"22,728.72"
3,4,fico_lt gt_fico x1 x2,"21,162.96"
4,4,fico fico_gt x1 x2,"19,045.94"
...,...,...,...
911,14,fico fico_gt fico_lt gt_fico x1 x10 x11 x13 x1...,"23,399.42"
912,14,fico fico_lt gt_fico x1 x10 x11 x12 x13 x14 x2...,"22,735.39"
913,14,fico_gt fico_lt gt_fico x1 x10 x11 x12 x13 x14...,"21,838.16"
914,14,fico fico_gt gt_fico x1 x10 x11 x12 x13 x14 x2...,"19,051.80"


In [264]:
%%time
df, y = create_synthetic_data_logistic(p=15)
df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x15": "gt_fico"}, inplace=True)
# df.rename(columns={"x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt", "x5": "gt_fico"}, inplace=True)

cands = df.columns[1:].tolist()
cands.remove('weight')
forced_features = ['x1', 'x9', 'x12', 'x11']
# forced_features=['x1', 'x2' , "*fico"]
forced_features =  ['x1', 'x2' , "*fico"]
print(forced_features)
res_weights, c , _ = bs_pkgs.best_subset(df, y, 100, start=2, stop=15,  candidates=cands,  forced_vars = forced_features,  method='logistic')
# res_weights, _ , _ = bs_pkgs.best_subset(df, y, 100, start=2, stop=15,  candidates=cands, method='logistic')

# res_weights, _ , _ = bs_pkgs.best_subset_orig(df, y, 2, start=2, stop=5,  candidates=cands)

res_weights = order_models_filed_all(res_weights)
res_weights = res_weights.reset_index(drop=True)
print(c)
res_weights

['x1', 'x2', '*fico']
Finished Var Family: 2  Skipped
# Models:  12 Finished Var Family: 3
# Models:  54 Finished Var Family: 4
# Models:  175 Finished Var Family: 5
# Models:  145 Finished Var Family: 6
# Models:  134 Finished Var Family: 7
# Models:  125 Finished Var Family: 8
# Models:  144 Finished Var Family: 9
# Models:  132 Finished Var Family: 10
# Models:  131 Finished Var Family: 11
# Models:  119 Finished Var Family: 12
# Models:  80 Finished Var Family: 13
# Models:  15 Finished Var Family: 14
# Models:  1 Finished Var Family: 15
1267
CPU times: total: 688 ms
Wall time: 218 ms


,Var Number,Models,Scores
0,3,fico x1 x2,"18,375.78"
1,3,gt_fico x1 x2,"16,810.12"
2,4,fico fico_lt x1 x2,"22,728.72"
3,4,fico_lt gt_fico x1 x2,"21,162.96"
4,4,fico fico_gt x1 x2,"19,045.94"
...,...,...,...
911,14,fico fico_gt fico_lt gt_fico x1 x10 x11 x13 x1...,"23,399.42"
912,14,fico fico_lt gt_fico x1 x10 x11 x12 x13 x14 x2...,"22,735.39"
913,14,fico_gt fico_lt gt_fico x1 x10 x11 x12 x13 x14...,"21,838.16"
914,14,fico fico_gt gt_fico x1 x10 x11 x12 x13 x14 x2...,"19,051.80"


In [265]:
compare_dataframes(exhaustive_filter,res_weights)

Dataset Match


# Test 2: Compare orig and new algo

In [261]:
%%time
df, y = create_synthetic_data_logistic(p=50)
# df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x15": "gt_fico"}, inplace=True)
# df.rename(columns={"x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt", "x5": "gt_fico"}, inplace=True)

cands = df.columns[1:].tolist()
cands.remove('weight')
forced_features = ['x1', 'x9', 'x12', 'x11']
 
print(forced_features)
res_weights, c , _ = bs_pkgs.best_subset(df, y, 2, start=2, stop=15,  candidates=cands,  forced_vars = forced_features,  method='logistic')
# res_weights, _ , _ = bs_pkgs.best_subset(df, y, 100, start=2, stop=15,  candidates=cands, method='logistic')

# res_weights, _ , _ = bs_pkgs.best_subset_orig(df, y, 2, start=2, stop=5,  candidates=cands)

res_weights = order_models_filed_all(res_weights)
res_weights = res_weights.reset_index(drop=True)
print(c)
res_weights

['x1', 'x9', 'x12', 'x11']
Finished Var Family: 2  Skipped
Finished Var Family: 3  Skipped
# Models:  1010 Finished Var Family: 4
# Models:  1571 Finished Var Family: 5
# Models:  1311 Finished Var Family: 6
# Models:  1083 Finished Var Family: 7
# Models:  2093 Finished Var Family: 8
# Models:  1618 Finished Var Family: 9
# Models:  4207 Finished Var Family: 10
# Models:  8552 Finished Var Family: 11
# Models:  14181 Finished Var Family: 12
# Models:  20965 Finished Var Family: 13
# Models:  34206 Finished Var Family: 14
# Models:  54366 Finished Var Family: 15
145163
CPU times: total: 12.7 s
Wall time: 17.9 s


,Var Number,Models,Scores
0,4,x1 x11 x12 x9,"6,102.14"
1,5,x1 x11 x12 x2 x9,"17,140.71"
2,5,x1 x11 x12 x4 x9,"10,391.73"
3,6,x1 x11 x12 x2 x4 x9,"21,414.85"
4,6,x1 x11 x12 x2 x3 x9,"18,700.71"
5,7,x1 x11 x12 x2 x3 x4 x9,"22,985.43"
6,7,x1 x11 x12 x2 x4 x5 x9,"22,076.14"
7,8,x1 x11 x12 x2 x3 x4 x5 x9,"23,626.71"
8,8,x1 x11 x12 x15 x2 x3 x4 x9,"22,988.87"
9,9,x1 x11 x12 x15 x2 x3 x4 x5 x9,"23,630.11"


In [260]:
%%time
df, y = create_synthetic_data_logistic(p=50)
# df.rename(columns ={ "x3": 'fico', "x4": 'fico_lt', "x5": "fico_gt",  "x5": "gt_fico"}, inplace=True)
cands = df.columns[1:].tolist()
cands.remove('weight')
 
# res_weights, _ , _ = bs_pkgs.best_subset(df, y, 100, start=2, stop=15,  candidates=cands,  forced_vars=['x1', 'x2', "*fico*"], method='logistic')
res_weights_orig, c , _ = bs_pkgs.best_subset_orig(df, y, 2, start=2, stop=15,  candidates=cands)

res_weights_orig = order_models_filed_all(res_weights_orig)
res_weights_orig = res_weights_orig.reset_index(drop=True)
print(c)
res_weights

# Models:  21 Finished Var Family:  2
# Models:  175 Finished Var Family:  3
# Models:  1010 Finished Var Family:  4
# Models:  1571 Finished Var Family:  5
# Models:  1311 Finished Var Family:  6
# Models:  1083 Finished Var Family:  7
# Models:  2093 Finished Var Family:  8
# Models:  1618 Finished Var Family:  9
# Models:  4232 Finished Var Family:  10
# Models:  8951 Finished Var Family:  11
# Models:  15366 Finished Var Family:  12
# Models:  23630 Finished Var Family:  13
# Models:  38041 Finished Var Family:  14
# Models:  60548 Finished Var Family:  15
159650
CPU times: total: 2min 19s
Wall time: 33.4 s


,Var Number,Models,Scores
0,4,x1 x11 x12 x9,"6,102.14"
1,5,x1 x11 x12 x2 x9,"17,140.71"
2,5,x1 x11 x12 x4 x9,"10,391.73"
3,6,x1 x11 x12 x2 x4 x9,"21,414.85"
4,6,x1 x11 x12 x2 x3 x9,"18,700.71"
5,7,x1 x11 x12 x2 x3 x4 x9,"22,985.43"
6,7,x1 x11 x12 x2 x4 x5 x9,"22,076.14"
7,8,x1 x11 x12 x2 x3 x4 x5 x9,"23,626.71"
8,8,x1 x11 x12 x15 x2 x3 x4 x9,"22,988.87"
9,9,x1 x11 x12 x15 x2 x3 x4 x5 x9,"23,630.11"


# Compare

In [262]:
compare_dataframes(res_weights_orig,res_weights)

Dataset Match
